## Reading Data

In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cd ASD && ls | wc -l
asd_files = os.listdir('/content/drive/MyDrive/Speech/FinalData/ASD/')

/bin/bash: line 0: cd: ASD: No such file or directory


In [ ]:
! cd TD && ls | wc -l
td_files = os.listdir('/content/drive/MyDrive/Speech/FinalData/TD/')

/bin/bash: line 0: cd: TD: No such file or directory


In [ ]:
import re
from tqdm import tqdm

In [ ]:
import numpy

In [ ]:
import nltk
nltk.download('punkt')
from collections import Counter

In [ ]:
sentence = 'i am preprocessing the text'
words = nltk.word_tokenize(sentence)

In [ ]:
cd /content/drive/MyDrive/Speech/FinalData/

/content/drive/MyDrive/Speech/FinalData


In [ ]:
df = pd.read_csv('childesData.csv', index_col = 0)

In [ ]:
df.head(10)

,ID,ChildText,Label,ChildTheraText
0,060622.cha,these are Brett's . | drop them off . | do wi...,0,what do you do ? | mix the stuff in . $ right...
1,030900.cha,want lunch . | want some lunch . | have some ...,0,roger want some lunch ? | want lunch . $ let'...
2,080506.cha,"hi , Helen . | hi Tina . | xxx . | hi Tina . ...",0,"okay ? | hi , Helen . $ did you say hi to Tin..."
3,080808.cha,nah . | don't do that . | no . | I don't care...,0,"yeah , come on . | don't do that . $ come on ..."
4,1025.cha,oh ‡ I found a jelly+bean . | I wanna eat it ...,1,you found a jelly+bean . | I wanna eat it . $...
5,040715.cha,"come . | ready , go . | &mm . | &mm . | doggy...",0,what do you say ? | come . $ open it . | dogg...
6,111.cha,&-um make a xxx and dump . | make dump . | bi...,1,a what ? | make dump . $ is that it ? | yeah ...
7,030806.cha,want water . | water . | sit down . | sit rig...,0,Stuart . | want water . $ you want water ? | ...
8,120.cha,beep@o beep@o . | &ah xxx . | &sh . | one . |...,0,"hm| &ah xxx . $ wow, yeah . | &sh . $ yeah, c..."
9,billy2.cha,ni(ght)_night . | ni(ght)_night . | are these...,1,where is the baby ? | ni(ght)_night . $ what ...


##ngrams

In [ ]:
import re
import os
try:
    import nltk
except:
    print("First install NLTK using pip install nltk command")
    exit()
try:
    import gensim
    from gensim import corpora, models, similarities
except:
    print("First install Gensim using pip install nltk command")
    exit()


class Stemmer:
    w2vModel = None
    sensitivity = 10

    # constructor
    def __init__(self, modelLocation= "w2vModel", w2vModel = None):
        try:
            self.w2vModel = gensim.models.Word2Vec.load(modelLocation)
        except:
            print("Could not locate the w2vModel file in the directory : "+(modelLocation))
            print("Try to load the w2vModel and try again")



    #  ----------- Stemming functions -----------


    # takes a word and removes the repeated occurance of characters in that word
    # outputs word without repeat consecutive occurance of the word
    def RepetitionStemmer(self, word):
        # find repeted occurence of letters in a word
        # remove the occurence
        i=0
        newWord = ''
        while(i <len(word)):
            c = word[i]
            newWord+=c
            while(i<len(word) and word[i] == c):
                i=i+1

        return newWord

    # takes a word2vec model, word and nWords(to run most similar on - higher the better but slower)
    # output the list of words similar to that word ( including that word passed through repetition stemmer)
    def WordEmbeddingStemmer(self, w2vModel, word, nWords = 10):

        try:
            similarWordsList =[w2vModel.wv.most_similar(word, topn = nWords )[i][0] for i in range(10)]
        except:
            return self.RepetitionStemmer(word)

        word = self.RepetitionStemmer(word)

        outputList = []
        for similarWord in similarWordsList:
            stemmSimilarWord = self.RepetitionStemmer(similarWord)
            w0 = word
            w1 = word[:-1]
            sw0 = stemmSimilarWord
            sw1 = stemmSimilarWord[:-1]

            if (sw0 in w0) or( w0 in sw0) or (sw1 in w0) or (w1 in sw0):
                if(len(stemmSimilarWord)<len(word)):
                    outputList.append(stemmSimilarWord)
                else:
                    outputList.append(word)
        if len(outputList) == 0:
            outputList.append(word)
        return outputList[0]

    # stemmers
    def stemWord(self, word):
        return self.WordEmbeddingStemmer(self.w2vModel, word)

    def stemListOfWords(self, listOfWords):
        return [self.WordEmbeddingStemmer(self.w2vModel, word) for word in listOfWords]

    def stem2dListOfWords(self, listOfWords2d):
        output = []
        for sentenceOfWords in listOfWords2d:
            output.append([self.WordEmbeddingStemmer(self.w2vModel, word) for word in sentenceOfWords])
        return output
myStemmer = Stemmer()

In [ ]:
def stem(sentence):
  words = sentence.split()
  for i in range(len(words)):
    words[i] = myStemmer.stemWord(words[i])
  return words


In [ ]:
df_hindi = pd.read_csv('childesData_hinglish.csv')

In [ ]:
from nltk.util import ngrams
import re

def n_grams(text):
  sentences = re.split(" .  ? ",text)
  uni=[]
  bi=[]
  for sentence in sentences:
    stemmed = stem(sentence)
    unigrams = ngrams(stemmed, 1)
    bigrams= ngrams(stemmed, 2)
    uni.extend(unigrams)
    bi.extend(bigrams)
  uni_sorted=sorted_grams(uni,1)
  bi_sorted=sorted_grams(bi,2)
  return uni_sorted, bi_sorted

def sorted_grams(ngrams, n=2):
    counts = {}
    for ngram in ngrams:
        counts[ngram] = counts.get(ngram, 0) + 1

    return sorted(((v,k) for k,v in counts.items()), reverse=True)

all_text = ''

for i in range(len(df_hindi['ChildText'])):
    if(type(df_hindi['ChildText'][i]) == float):
        df_hindi['ChildText'][i] = ""
    cur = []
    cunits = list(df_hindi['ChildText'][i].split('.'))
    tunits = list(df_hindi['ChildTheraText'][i].split('$'))
    text = df_hindi['ChildText'][i].replace('.', '').replace('  ',' ')
    all_text += text

uni_sorted,bi_sorted= n_grams(all_text)
print(uni_sorted)
print(bi_sorted)

In [ ]:
## unigrams = hai, ha, h, ye, nahin, aur #frequency above 40
## bigrams = (ha, ha), (hota, h), (ho, gya), (yeh, rhi), (raha, hai) #frequency above 10

In [ ]:
ngram_featureset=[]
for i in range(len(df_hindi['ChildText'])):
    counts=[0,0,0,0,0,0,0,0,0,0,0]
    if(type(df_hindi['ChildText'][i]) == float):
        df_hindi['ChildText'][i] = ""
    cur = []
    cunits = list(df_hindi['ChildText'][i].split('.'))
    tunits = list(df_hindi['ChildTheraText'][i].split('$'))
    text = df_hindi['ChildText'][i].replace('.', '').replace('  ',' ')
    uni_sorted,bi_sorted= n_grams(text)
    for unigram in uni_sorted:
      if (unigram[1][0]=='se'):
        counts[0] = unigram[0]
      if (unigram[1][0]=='aur'):
        counts[1] = unigram[0]
      if (unigram[1][0]=='aise'):
        counts[2] = unigram[0]
      if ((unigram[1][0]=='ye' or (unigram[1][0]=='yeh'))):
        counts[3] = unigram[0]
      if (unigram[1][0]=='nahin'):
        counts[4] = unigram[0]
    for bigram in bi_sorted:
      if ((bigram[1][0]=='ha') and (bigram[1][1]=='ha')):
        counts[5] = bigram[0]
      if ((bigram[1][0]=='hota') and (bigram[1][1]=='hai')):
        counts[5] = bigram[0]
      if ((bigram[1][0]=='ho') and (bigram[1][1]=='gya')):
        counts[5] = bigram[0]
      if ((bigram[1][0]=='yeh') and (bigram[1][1]=='rhi')):
        counts[5] = bigram[0]
      if ((bigram[1][0]=='rha') and (bigram[1][1]=='hai')):
        counts[5] = bigram[0]
    counts[10] = df_hindi['Label'][i]
    ngram_featureset.append(counts)
print(ngram_featureset[:][:])

In [ ]:
pd.DataFrame(ngram_featureset).to_csv('features_ngram.csv')

##opensmile


In [ ]:
!pip install opensmile

In [ ]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    sampling_rate = 44100
)

In [ ]:
data = pd.DataFrame()
#axis0
#pd.concat([df1, df2], axis=0)
files = ['0','1','2','3','4','5','6','7','8','9','10','12','15','16','17','18']
for fname in files:
  y = smile.process_file('ASD_hinglish/' + str(fname)+'child.wav')
  y["label"] = [0]
  if (fname == '0'):
    data = y
  else:
    data = pd.concat([data,y], axis = 0)
  data.head()

files = ['535','536','537','538','539','540','541','542','544','545','550','551','552','553','554','594','9531','9532']
for fname in files:
  y = smile.process_file('TD_hinglish/' + str(fname)+'child.wav')
  y["label"] = [1]
  data = pd.concat([data,y], axis = 0)

In [ ]:
data.to_csv('features_acoustic.csv')

##feature reduction

In [ ]:
data = pd.read_csv("features_acoustic.csv")

In [ ]:
data = data.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2','file','start','end'],axis =1)

In [ ]:
from sklearn.model_selection import KFold

# Assuming you have your data in a pandas DataFrame called 'df'
np.random.seed(42)

# Specify the number of folds
num_folds = 5

# Initialize the cross-validator
kf = KFold(n_splits=num_folds, random_state=42, shuffle = True)

# Create lists to store the training and testing datasets for each fold
train_datasets = []
test_datasets = []

# Split the data into folds and store the indices
for train_index, test_index in kf.split(data):
    # Obtain the training and testing datasets based on the indices
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Append the datasets to the lists
    train_datasets.append(train_data)
    test_datasets.append(test_data)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    plt.figure(figsize=(90,90))
    heatmap = sns.heatmap(df.corr(), vmin=-1,
                      vmax=1, annot=True)
    plt.title("Spearman Correlation")
    return(r)

correlation = display_correlation(data)

In [ ]:
spearman = correlation.sort_values('label')['label']
one = (spearman[0:16]).to_frame()
two = (spearman[-2:]).to_frame()
selected_features_list = pd.concat([one, two], axis = 0)

In [ ]:
selected_features = list(selected_features_list.index)
print(spearman)

In [ ]:
selected_features

In [ ]:
data_new = data[['unintell_prop',
 'discourse2',
 'unexpected_words',
 'avg_sentence',
 'ADP',
 'ADJ']]

In [ ]:
data_new.to_csv('text_selected.csv')

###Fold1

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[0])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_1 = list(selected_features_list.index)
print(selected_features_1)

In [ ]:
data_1 = data[selected_features_1]

In [ ]:
data_1.to_csv('acoustic_1.csv')

###Fold2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[1])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_2 = list(selected_features_list.index)
print(selected_features_2)

In [ ]:
print(len(selected_features_2))

In [ ]:
data_2 = data[selected_features_2]

In [ ]:
data_2.to_csv('acoustic_2.csv')

###Fold3

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[2])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_3 = list(selected_features_list.index)
print(selected_features_3)

In [ ]:
data_3 = data[selected_features_3]

In [ ]:
data_3.to_csv('acoustic_3.csv')

###Fold4

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[3])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_4 = list(selected_features_list.index)
print(selected_features_4)

In [ ]:
data_4 = data[selected_features_4]

In [ ]:
data_4.to_csv('acoustic_1.csv')

###Fold5

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[4])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_5 = list(selected_features_list.index)
print(selected_features_5)

In [ ]:
data_5 = data[selected_features_5]

In [ ]:
data_5.to_csv('acoustic_5.csv')

##Text Folds

In [ ]:
data = pd.read_csv("features_hindi_labelled.csv")

In [ ]:
data = data.drop(['Unnamed: 0'],axis = 1)

In [ ]:
from sklearn.model_selection import KFold

np.random.seed(42)

num_folds = 5

kf = KFold(n_splits=num_folds, random_state=42, shuffle = True)

train_datasets = []
test_datasets = []

for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]
    train_datasets.append(train_data)
    test_datasets.append(test_data)

###Folds definition

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt

def display_correlation(df):
    r = df.corr(method="spearman")
    # plt.figure(figsize=(90,90))
    # heatmap = sns.heatmap(df.corr(), vmin=-1,
    #                   vmax=1, annot=True)
    # plt.title("Spearman Correlation")
    return(r)
correlation = display_correlation(train_datasets[4])

In [ ]:
spearman = correlation.sort_values('label')['label']
for i in range(len(spearman)):
  if spearman[i]<0:
    spearman[i] = -spearman[i]
spearman = spearman.sort_values()
print(spearman)
two = (spearman[-16:]).to_frame()
selected_features_list = two

In [ ]:
selected_features_1 = list(selected_features_list.index)
print(selected_features_1)

In [ ]:
data_1 = data[selected_features_1]

In [ ]:
data_1.to_csv('acoustic_1.csv')

##Model and data preparation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
temp = pd.read_csv("acoustic_1.csv")

In [ ]:
acoustic1 = (pd.read_csv("acoustic_1.csv").to_numpy())[:,1:]

In [ ]:
text = pd.read_csv('text_selected.csv').to_numpy()
ngram = pd.read_csv("features_ngram.csv").to_numpy()
acoustic = pd.read_csv("features_acoustic_selected.csv").to_numpy()
acoustic1 = (pd.read_csv("acoustic_1.csv").to_numpy())[:,1:]
acoustic2 = (pd.read_csv("acoustic_2.csv").to_numpy())[:,1:]
acoustic3 = (pd.read_csv("acoustic_3.csv").to_numpy())[:,1:]
acoustic4 = (pd.read_csv("acoustic_4.csv").to_numpy())[:,1:]
acoustic5 = (pd.read_csv("acoustic_5.csv").to_numpy())[:,1:]

label = ngram[:,-1]
text_labels =text
text = text[:, 0:-1]
n_gram = ngram[:,1:-1]
acoustic = acoustic[:,1:-1]

sc = StandardScaler()
pca = PCA(n_components=10)
pca_ = PCA(n_components= 5)

# sc.fit(acoustic)
# acoustic = sc.transform(acoustic)
# sc.fit(text)
# text = sc.transform(text)
# sc.fit(n_gram)
# n_gram = sc.transform(n_gram)


# pca.fit(acoustic)
# acoustic = pca.transform(acoustic)
# pca_.fit(text)
# text = pca_.transform(text)
# pca_.fit(n_gram)
# n_gram = pca_.transform(n_gram)

In [ ]:
svm = SVC(kernel='linear', random_state = 0, C=1)
ann = MLPClassifier(max_iter=1000, activation='tanh', solver='sgd',alpha=0.7)
kmeans = KNeighborsClassifier(n_neighbors = 9)
rf = RandomForestClassifier(n_estimators = 30, random_state = 0)
lr = LogisticRegression(random_state=0, max_iter = 1000)
scoring = ['precision_macro', 'recall_macro','f1_macro']

##Model Training

In [ ]:
acoustic_rf = []
acoustic_svm = []
acoustic_lr = []
acoustic_kmeans = []

In [ ]:
from sklearn.model_selection import KFold

# Assuming you have your data in a pandas DataFrame called 'df'
np.random.seed(42)

# Specify the number of folds
num_folds = 5

# Initialize the cross-validator
kf = KFold(n_splits=num_folds, random_state=42, shuffle = True)

# Create lists to store the training and testing datasets for each fold
train_datasets = []
test_datasets = []

data = np.concatenate((text,acoustic), axis = 1)

data1 = np.concatenate((text,acoustic1), axis = 1)
data2 = np.concatenate((text,acoustic2), axis = 1)
data3 = np.concatenate((text,acoustic3), axis = 1)
data4 = np.concatenate((text,acoustic4), axis = 1)
data5 = np.concatenate((text,acoustic5), axis = 1)

# data = acoustic

# data1 = acoustic1
# data2 = acoustic2
# data3 = acoustic3
# data4 = acoustic4
# data5 = acoustic5

# labels = np.reshape(ngram[:,-1], (-1, 1))
# data1 = np.concatenate((text,labels), axis = 1)
# data2 = np.concatenate((text,labels), axis = 1)
# data3 = np.concatenate((text,labels), axis = 1)
# data4 = np.concatenate((text,labels), axis = 1)
# data5 = np.concatenate((text,labels), axis = 1)

# labels = np.reshape(ngram[:,-1], (-1, 1))
# data1 = np.concatenate((n_gram,labels), axis = 1)
# data2 = np.concatenate((n_gram,labels), axis = 1)
# data3 = np.concatenate((n_gram,labels), axis = 1)
# data4 = np.concatenate((n_gram,labels), axis = 1)
# data5 = np.concatenate((n_gram,labels), axis = 1)

# data = np.concatenate((n_gram, text,acoustic), axis = 1)

# data1 = np.concatenate((n_gram, text,acoustic1), axis = 1)
# data2 = np.concatenate((n_gram, text,acoustic2), axis = 1)
# data3 = np.concatenate((n_gram, text,acoustic3), axis = 1)
# data4 = np.concatenate((n_gram, text,acoustic4), axis = 1)
# data5 = np.concatenate((n_gram, text,acoustic5), axis = 1)

# Split the data into folds and store the indices
fold = 1
for train_index, test_index in kf.split(data):
    # Obtain the training and testing datasets based on the indices
    if fold==1:
      train_data = data1[train_index]
      test_data = data1[test_index]

    if fold==2:
      train_data = data2[train_index]
      test_data = data2[test_index]

    if fold==3:
      train_data = data3[train_index]
      test_data = data3[test_index]

    if fold==4:
      train_data = data4[train_index]
      test_data = data4[test_index]

    if fold==5:
      train_data = data5[train_index]
      test_data = data5[test_index]

    # Append the datasets to the lists
    train_datasets.append(train_data)
    test_datasets.append(test_data)
    fold += 1

In [ ]:
scores = []
results_arr = []
for fold in range(5):
  rf.fit(train_datasets[fold][:,:-1], train_datasets[fold][:,-1])
  results = metrics.classification_report(test_datasets[fold][:,-1], rf.predict(test_datasets[fold][:,:-1]))
  results_arr.append(results)
  f1_score_macro = float(results.split()[21])
  scores.append(f1_score_macro)
  y_pred_simple = rf.predict(test_datasets[fold][:,:-1])
  baseline_accuracy = bootstrap_score(test_datasets[fold][:,-1], y_pred_simple, metric=macro_f1, random_state=5 )
  print(baseline_accuracy)
print(scores)
print(np.mean(scores))

In [ ]:
#!pip install statkit
from sklearn.metrics import f1_score
from statkit.non_parametric import bootstrap_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.utils import resample

# Define the macro F1 score as a custom scorer
macro_f1_score = make_scorer(f1_score, average='macro')

def macro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')


In [ ]:
results = results_arr[3]
print(results)
print(float(results.split()[21]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Prepare your data
f1_scores = scores
std_deviations = [np.std(scores),np.std(scores),np.std(scores),np.std(scores),np.std(scores)]

# Set up the plot
fig, ax = plt.subplots()

# Define the x-axis positions for the bars
x_pos = np.arange(len(f1_scores))

# Plot the bars
ax.bar(x_pos, f1_scores, yerr=std_deviations, align='center', alpha=0.5, ecolor='black', capsize=10)

# Customize the plot
ax.set_xlabel('Fold')
ax.set_ylabel('F1 Score')
ax.set_title('F1 Scores for each fold')
ax.set_xticks(x_pos)
ax.set_xticklabels(['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5'])
ax.yaxis.grid(True)

# Show the plot
plt.tight_layout()
plt.show()


##Shap Analysis

In [ ]:
text = pd.read_csv('features_hindi_labelled.csv').to_numpy()

In [ ]:
text = pd.read_csv('features_all_english.csv').to_numpy()

In [ ]:
label = text[:,-1]

In [ ]:
data = text[:, 1:-1]
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=0)

In [ ]:
scores = cross_validate(rf, x_train, y_train, cv=5,scoring=scoring)
print('Precision macros:', np.mean(scores['test_precision_macro']))
print('Recall macros:', np.mean(scores['test_recall_macro']))
print('F1 macros:', np.mean(scores['test_f1_macro']))

In [ ]:
import shap
rf.fit(x_train, y_train)
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(x_train)
shap.summary_plot(shap_values[0], x_train, feature_names = ['mlum','ndwr','avg_sentence','num_sentences','unintell_prop','repProp','ecolalia','discourse1','discourse2','unexpected_words','metaphors','num_questions','ADJ','ADP','ADV','AUX','CCONJ','DET','INTJ','NOUN','NUM','PART','PRON','PROPN','SCONJ','VERB'])

In [ ]:
svm.fit(x_train, y_train)
print(metrics.classification_report(y_test, svm.predict(x_test)))

In [ ]:
svm_explainer = shap.KernelExplainer(svm.predict, x_train)
svm_shap_values = svm_explainer.shap_values(x_train)
shap.summary_plot(svm_shap_values, x_train, feature_names = ['unintell_prop','discourse2','unexpected_words','avg_sentence','ADP','ADJ'])

In [ ]:
scores = cross_validate(ann, x_train, y_train, cv=LeaveOneOut(),scoring=scoring)
print('Precision macros:', np.mean(scores['test_precision_macro']))
print('Recall macros:', np.mean(scores['test_recall_macro']))
print('F1 macros:', np.mean(scores['test_f1_macro']))
ann.fit(x_train, y_train)
print(metrics.classification_report(y_test, ann.predict(x_test)))

In [ ]:
scores = cross_validate(rf, x_train, y_train, cv=LeaveOneOut(),scoring=scoring)
print('Precision macros:', np.mean(scores['test_precision_macro']))
print('Recall macros:', np.mean(scores['test_recall_macro']))
print('F1 macros:', np.mean(scores['test_f1_macro']))
rf.fit(x_train, y_train)
print(metrics.classification_report(y_test, rf.predict(x_test)))

In [ ]:
!pip install shap
import shap

In [ ]:
svm_explainer = shap.KernelExplainer(svm.predict, x_train)
svm_shap_values = svm_explainer.shap_values(x_train)
shap.summary_plot(svm_shap_values, x_train, feature_names = ['spectralFlux_sma3_amean','loudness_sma3_stddevRisingSlope','loudness_sma3_percentile80.0','loudness_sma3_percentile20.0','loudness_sma3_amean','loudness_sma3_meanFallingSlope','loudness_sma3_stddevFallingSlope','equivalentSoundLevel_dBp','spectralFluxV_sma3nz_amean','spectralFluxUV_sma3nz_amean','loudness_sma3_percentile50.0','loudness_sma3_meanRisingSlope','loudness_sma3_pctlrange0-2','mfcc1_sma3_stddevNorm','mfcc1V_sma3nz_stddevNorm','MeanUnvoicedSegmentLength','mfcc1_sma3_amean'])

In [ ]:
scores = cross_validate(kmeans, x_train, y_train, cv=LeaveOneOut(),scoring=scoring)
print('Precision macros:', np.mean(scores['test_precision_macro']))
print('Recall macros:', np.mean(scores['test_recall_macro']))
print('F1 macros:', np.mean(scores['test_f1_macro']))
kmeans.fit(x_train, y_train)
print(metrics.classification_report(y_test, kmeans.predict(x_test)))